# Протоколы аутентификации

In [54]:
IS_DEBUG = 1

In [1]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

## Протокол PAP
![PAP Protocol](./images/PAP_1.png)

-----

In [2]:
# Использую хэш-функцию ГОСТ 34.11-2012 "Стрибог"
from pygost.gost34112012256 import GOST34112012256

# Эта либа тоже понадобится
import binascii

# Для генерации случайных чисел (КГПСЧ)
from Crypto.Random import get_random_bytes

------

In [131]:
class PAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login: str, password: str):
        trace('[PAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[PAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Рассчитаю хэш пароля и запишу в базу
        #
        
        hasher = GOST34112012256()
        hasher.update(password.encode())
        
        self._db[login] = hasher.digest()
        trace('[PAP Server]', f'User successfully registered, password hash: {hasher.hexdigest()}')
        
    def login(self, login: str, password: str) -> bool:
        trace('[PAP Server]', f'Attempt to login with credentials {login}:{password}')
        
        if login not in self._db:
            trace('[PAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, рассчитаю хэш пароля и сравню с имеющимся
        #
        
        hasher = GOST34112012256()
        hasher.update(password.encode())
        
        real_hash = self._db[login]
        
        trace('[PAP Server]', f'Password hash: {hasher.hexdigest()}')
        trace('[PAP Server]', f'Stored hash:   {binascii.hexlify(real_hash).decode()}')
        
        return hasher.digest() == real_hash

In [132]:
class PAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[PAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: PAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: PAPServer):
        if srv.login(self._login, self._passwd):
            print('[PAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[PAP Client]', f'Wrong username or password')

----

In [133]:
# Создаю сервер
server = PAPServer()

In [134]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = PAPClient('Alice', 'Trust_m3_0r_n0t')
eve = PAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [PAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [PAP Client] User with credentials Alice:try_t0_gu3$$ created


In [135]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [PAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] User successfully registered, password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8


In [136]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] Password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] User Alice logged in successfully


In [137]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:try_t0_gu3$$
[TRACE] [PAP Server] Password hash: 034eda071f984d17f28f9ae214c5d9b9a9fc0e112df98d0e4b5b2eb3d701b314
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] Wrong username or password


-----

## Протокол CHAP
![CHAP Protocol](./images/CHAP_1.png)

In [138]:
class CHAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login, password):
        trace('[CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        trace('[CHAP Server]', f'User successfully registered')
    
    def login(self, login, response):
        trace('[CHAP Server]', f'''Attempt to login with: {login = }, 
        response = {binascii.hexlify(response).decode()}''')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[CHAP Server]', f'No challenge was generated for {login = }')
            return False
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        hasher = GOST34112012256()
        hasher.update(challenge)
        hasher.update(self._db[login][0])
        
        trace('[CHAP Server]', f'Calculated hash: {hasher.hexdigest()}')
        trace('[CHAP Server]', f'Received hash:   {binascii.hexlify(response).decode()}')
        
        return hasher.digest() == response
    
    def generate_challenge(self, login):
        trace('[CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return None
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[CHAP Server]', f'Generated challenge = {binascii.hexlify(challenge).decode()}')
        return challenge

In [139]:
class CHAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: CHAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: CHAPServer):
        challenge = srv.generate_challenge(self._login)
        if challenge is None:
            print('[CHAP Client]', f'Wrong username or password')
            return
        
        hasher = GOST34112012256()
        hasher.update(challenge)
        hasher.update(self._passwd.encode())
        
        if srv.login(self._login, hasher.digest()):
            print('[CHAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[CHAP Client]', f'Wrong username or password')

----

In [140]:
# Создаю сервер
server = CHAPServer()

In [141]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = CHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = CHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [142]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [CHAP Server] User successfully registered


In [143]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = f73764ce2919d1df8367cca554c11cdc
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = 7c0607492005491126fde96805667c39c9447f4e243c92f609027b2167e65c62
[TRACE] [CHAP Server] Calculated hash: 7c0607492005491126fde96805667c39c9447f4e243c92f609027b2167e65c62
[TRACE] [CHAP Server] Received hash:   7c0607492005491126fde96805667c39c9447f4e243c92f609027b2167e65c62
[CHAP Client] User Alice logged in successfully


In [144]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = 75b9afc1d9c7e83bcf444aa6eb74af9a
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = f9f6e06365f122d25466919c06ce0a8bc95a583779d555b7cea2582383c5285f
[TRACE] [CHAP Server] Calculated hash: 98f34e3c4aa8e59db8e8ebb1cac3047f98ac69e2f3bac321f5330747ca3c05c0
[TRACE] [CHAP Server] Received hash:   f9f6e06365f122d25466919c06ce0a8bc95a583779d555b7cea2582383c5285f
[CHAP Client] Wrong username or password


----

## Двустронний протокол CHAP

In [145]:
class ModifiedCHAPServer(object):
    def __init__(self):
        self._db = {}
        self._server_login = 'Alice'
        self._server_pass  = '1qazXSW@'
        
    def register_user(self, login, password):
        trace('[Modified CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[Modified CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        
        trace('[Modified CHAP Server]', f'User successfully registered')
        
        return self._server_login, self._server_pass
        
    def login(self, login, response, client_challenge):
        trace(f'''[Modified CHAP Server]', f'Attempt to login with: {login = },
        response = {binascii.hexlify(response).decode()}, 
        client_challenge = {binascii.hexlify(client_challenge).decode()}''')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[Modified CHAP Server]', f'No challenge was generated for {login = }')
            return None
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        hasher = GOST34112012256()
        hasher.update(challenge)
        hasher.update(self._db[login][0])
        
        trace('[Modified CHAP Server]', f'Calculated hash: {hasher.hexdigest()}')
        trace('[Modified CHAP Server]', f'Received hash:   {binascii.hexlify(response).decode()}')
        
        if hasher.digest() != response:
            return None
        
        #
        # Теперь считаю хэш для челленджа клиента и возвращаю его
        #
        
        hasher = GOST34112012256()
        hasher.update(client_challenge)
        hasher.update(self._server_pass.encode())
        
        return self._server_login, hasher.digest()
    
    def generate_challenge(self, login):
        trace('[Modified CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[Modified CHAP Server]', f'Generated challenge = {binascii.hexlify(challenge).decode()}')
        
        return challenge

In [152]:
class ModifiedCHAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        self._servers_db = {}
        
        trace('[Modified CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: CHAPServer):
        try:
            login, password = srv.register_user(self._login, self._passwd)
            self._servers_db[login] = password
        except Exception as e:
            trace(e)
    
    def login(self, srv: CHAPServer):
        srv_challenge = srv.generate_challenge(self._login)
        if srv_challenge is None:
            print('[Modified CHAP Client]', f'Wrong username or password')
            return
        
        hasher = GOST34112012256()
        hasher.update(srv_challenge)
        hasher.update(self._passwd.encode())
        
        challenge = self.generate_challenge()
        srv_data  = srv.login(self._login, hasher.digest(), challenge)
        
        if srv_data is not None:
            srv_login, srv_response = srv_data
            
            if srv_login not in self._servers_db:
                print('[Modified CHAP Client]', f'Server {srv_login} not registered')
                return
            
            hasher = GOST34112012256()
            hasher.update(challenge)
            hasher.update(self._servers_db[srv_login].encode())
            
            trace('[Modified CHAP Client]', f'Calculated hash: {hasher.hexdigest()}')
            trace('[Modified CHAP Client]', f'Received hash:   {binascii.hexlify(srv_response).decode()}')
            
            if hasher.digest() == srv_response:
                print('[Modified CHAP Client]', f'User {self._login} logged in successfully')
                return
        
        print('[Modified CHAP Client]', f'Wrong username or password')
            
    @staticmethod
    def generate_challenge():
        return get_random_bytes(16)

------

In [153]:
# Создаю сервер
server = ModifiedCHAPServer()

In [154]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = ModifiedCHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = ModifiedCHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [Modified CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [Modified CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [155]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [Modified CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [Modified CHAP Server] User successfully registered


In [156]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = c993612a5b665465f889ca9017cae5ce
[TRACE] [Modified CHAP Server]', f'Attempt to login with: login = 'Alice',
        response = 7914eae9d05ae700b27bcf09dbd0cf40c28c75ea6fda8287ace7cc458a090f8a, 
        client_challenge = 4402dbf2f1bde9ec3fd8fb8361ebbd02
[TRACE] [Modified CHAP Server] Calculated hash: 7914eae9d05ae700b27bcf09dbd0cf40c28c75ea6fda8287ace7cc458a090f8a
[TRACE] [Modified CHAP Server] Received hash:   7914eae9d05ae700b27bcf09dbd0cf40c28c75ea6fda8287ace7cc458a090f8a
[TRACE] [Modified CHAP Client] Calculated hash: f5f43ff4bf4c9e551b2a66a7336b356941a78ee39c0659f69931d969ac207811
[TRACE] [Modified CHAP Client] Received hash:   f5f43ff4bf4c9e551b2a66a7336b356941a78ee39c0659f69931d969ac207811
[Modified CHAP Client] User Alice logged in successfully


In [157]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = 2554a7efd40c413c3f2a7efa98c4fb7d
[TRACE] [Modified CHAP Server]', f'Attempt to login with: login = 'Alice',
        response = 971b2f10c80346fcda0dd8a6d21150ea025e4d3c6d03e07f981326e391c88aa3, 
        client_challenge = 8453d89a87da400a5bbb77cb4864e232
[TRACE] [Modified CHAP Server] Calculated hash: 25332bf11a589da677824882d4aa07a9903262da72541ca891e07d5c6373897b
[TRACE] [Modified CHAP Server] Received hash:   971b2f10c80346fcda0dd8a6d21150ea025e4d3c6d03e07f981326e391c88aa3
[Modified CHAP Client] Wrong username or password


----

## Протокол S/KEY
![S/KEY Protocol](./images/SKEY_1.png)

In [18]:
# To be implemented